# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)


/home/lsossai/dev_pessoal/nd-data-eng-project-2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
     
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))
# print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
CASSANDRA_DB = "sparkifydb"
REPLICATION_OPTIONS = {'class' : 'SimpleStrategy',
                       'replication_factor': 1}
CREATE_KEYSPACE_QUERY = f"""
        CREATE KEYSPACE IF NOT EXISTS {CASSANDRA_DB}
        WITH REPLICATION = {REPLICATION_OPTIONS}
    """
    
try:
    session.execute(CREATE_KEYSPACE_QUERY)
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace(CASSANDRA_DB)
except Exception as e:
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
file = 'event_datafile_new.csv'

ARTIST_COLUMN = 0
FIRSTNAME_COLUMN = 1
ITEMINSESSION_COLUMN = 3
LASTNAME_COLUMN = 4
LENGHT_COLUMN = 5
SESSIONID_COLUMN = 8
SONG_COLUMN = 9
USERID_COLUMN = 10

In [9]:
## Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
CREATE_TABLE_QUERY1 = """CREATE TABLE IF NOT EXISTS session_item
                      (
                        sessionId     INT,
                        itemInSession INT,
                        artist        TEXT,
                        song          TEXT,
                        length        FLOAT,
                        PRIMARY KEY (sessionId, itemInSession)
                      ) """
  
try:
    session.execute(CREATE_TABLE_QUERY1)
except Exception as e:
    print(e)
                    

In [10]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length) "
        query += " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query, (int(line[SESSIONID_COLUMN]),
                                int(line[ITEMINSESSION_COLUMN]),
                                line[ARTIST_COLUMN], 
                                line[SONG_COLUMN], 
                                float(line[LENGHT_COLUMN])) )

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
SELECT_QUERY1 = """SELECT artist,
                        song,
                        length
                   FROM   session_item
                   WHERE  sessionId = 338
                          AND itemInSession = 4 """
try:
    rows = session.execute(SELECT_QUERY1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
CREATE_TABLE_QUERY2 = """CREATE TABLE IF NOT EXISTS user_session
                      (
                        userId        INT,
                        sessionId     INT,
                        artist        TEXT,
                        song          TEXT,
                        itemInSession INT,
                        firstName     TEXT,
                        lastName      TEXT,
                        PRIMARY KEY ((sessionId, userId), itemInSession)
                      ) 
                        WITH CLUSTERING ORDER BY (itemInSession ASC)
                      """
  
try:
    session.execute(CREATE_TABLE_QUERY2)
except Exception as e:
    print(e)


In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userId, sessionId, artist, song, itemInSession, firstName, lastName) "
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[USERID_COLUMN]),
                                int(line[SESSIONID_COLUMN]),
                                line[ARTIST_COLUMN],
                                line[SONG_COLUMN],
                                int(line[ITEMINSESSION_COLUMN]),
                                line[FIRSTNAME_COLUMN],
                                line[LASTNAME_COLUMN] ))

In [14]:
SELECT_QUERY2 = """SELECT artist,
                        song,
                        firstname,
                        lastname
                    FROM   user_session
                    WHERE  sessionid = 182
                        AND userid = 10"""
try:
    rows = session.execute(SELECT_QUERY2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
CREATE_TABLE_QUERY3 = """CREATE TABLE IF NOT EXISTS users_song
                      (
                        song          TEXT,
                        userId        INT,
                        firstName     TEXT,
                        lastName      TEXT,
                        PRIMARY KEY ((song), userId)
                      ) """
  
try:
    session.execute(CREATE_TABLE_QUERY3)
except Exception as e:
    print(e)                   

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_song (song, userId, firstName, lastName) "
        query += " VALUES (%s, %s, %s, %s) "
        session.execute(query, (line[SONG_COLUMN],
                                int(line[USERID_COLUMN]),
                                line[FIRSTNAME_COLUMN],
                                line[LASTNAME_COLUMN] ))

In [17]:
SELECT_QUERY3 = """SELECT firstname,
                          lastname
                    FROM   users_song
                    WHERE  song = 'All Hands Against His Own'"""
try:
    rows = session.execute(SELECT_QUERY3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
session.execute(f"DROP TABLE IF EXISTS {CASSANDRA_DB}.session_item")
session.execute(f"DROP TABLE IF EXISTS {CASSANDRA_DB}.user_session")
session.execute(f"DROP TABLE IF EXISTS {CASSANDRA_DB}.user_song")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()